In [77]:
import torch
import scanpy as sc
import numpy as np
import h5py
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from normalising_flows import target

In [2]:
adata = sc.read("processed_files/reduced_features_RNA_Velocity.h5ad")

In [7]:
x_data = adata.X.toarray()

In [29]:
labels = (adata.obs["disease"] == "Alzheimer disease").values

In [30]:
labels

array([False, False, False, ...,  True,  True,  True], shape=(282930,))

In [37]:
file = h5py.File("processed_files/processed_dataset.h5", "w")
file.create_dataset("genes", data = x_data)
file.create_dataset("labels", data = labels)
file.close()

In [57]:
loaded_data = h5py.File("processed_files/processed_dataset.h5", "r")

In [58]:
a = loaded_data["genes"]
loaded_data.close()

In [49]:
a.shape

(282930, 2500)

In [72]:
from torch.utils.data import Dataset, DataLoader

class Alzheimers_Dataset(Dataset):
    def __init__(self):
        self.file = h5py.File("processed_files/processed_dataset.h5", "r")
        self.genes = self.file["genes"]
        self.labels = self.file["labels"]
        self.length = self.genes.shape[0]

    def __len__(self):
        return self.length

    def __getitem__(self, idx):
        return self.genes[idx], self.labels[idx].astype(int)

In [75]:
dataset = Alzheimers_Dataset()
dataloader = DataLoader(dataset, batch_size=512, shuffle=True)

In [84]:
(a,b) = next(iter(dataloader))
a

tensor([[0.0000, 1.4152, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 1.1327, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]])

In [91]:
class classifier_model(nn.Module):
    def __init__(self, gene_shape):
        super(classifier_model, self).__init__()
        self.linear = nn.Linear(gene_shape, 1)

    def forward(self, x):
        return torch.sigmoid(self.linear(x))

In [ ]:
model = classifier_model(2500)
optimizer = optim.Adam(model.parameters(), lr=3e-4)
loss = torch.nn.BCELoss()

epochs = 10
losses = []

for epoch in range(epochs):
    for i, (data, labels) in enumerate(dataloader):
        optimizer.zero_grad()
        pred = model(data)
        computed_loss = loss(torch.squeeze(pred), labels.float())
        computed_loss.backward()
        optimizer.step()
        if i % 100 == 0:
            print(computed_loss.item())
            losses.append(computed_loss.item())

0.6851346492767334
0.5759737491607666
0.5118807554244995
0.5058585405349731
0.5291810035705566
0.5005697011947632
0.5237830281257629
0.4192688465118408
0.4540809988975525
0.4411153793334961


In [104]:
labels.float()

tensor([1., 1., 1., 1., 0., 1., 0., 1., 0., 1., 1., 1., 1., 0., 0., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1.,
        1., 0., 0., 1., 1., 0., 1., 1., 1., 1., 0., 1., 0., 1., 1., 1., 0., 0.,
        1., 1., 0., 1., 1., 0., 1., 0., 1., 1., 1., 0., 0., 1., 1., 0., 1., 0.,
        1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 1., 1., 1.,
        1., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 1., 1.,
        1., 0., 1., 1., 1., 0., 1., 1., 1., 1., 1., 0., 1., 0., 1., 1., 0., 0.,
        1., 1., 1., 1., 1., 0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 1., 1., 1., 1., 0., 1., 1.,
        1., 1., 1., 1., 1., 1., 0., 0., 0., 1., 1., 1., 0., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 0., 1., 1., 1., 1., 0., 1., 1.,
        1., 1., 0., 0., 1., 1., 1., 1., 

In [94]:
labels

tensor([0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0,
        0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1,
        1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1,
        1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
        1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0,
        1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1,
        0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0,
        0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1,
        1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0,
        0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1,
        1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1,
        1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1,